In [2]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [3]:
train = pd.read_csv('../Datasets/SPECT Heart/spect_train_binary.csv')
test = pd.read_csv('../Datasets/SPECT Heart/spect_test_binary.csv')

In [4]:
X_train = train.drop('OVERALL_DIAGNOSIS', axis=1)
y_train = train['OVERALL_DIAGNOSIS']
X_test = test.drop('OVERALL_DIAGNOSIS', axis=1)
y_test = test['OVERALL_DIAGNOSIS']


# AUTOML

In [9]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

In [10]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.8128342245989305

In [11]:
import pickle
with open('./models/spect_heart_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

# RANDOM FOREST

In [12]:
import sklearn.ensemble

model = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=42)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [13]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.7647058823529411

# SVC

In [13]:
import sklearn.svm
import sklearn.model_selection

model = sklearn.svm.SVC(random_state=42)
params = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['linear', 'ploy', 'rbf', 'sigmoid'], 'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
gridSearch = sklearn.model_selection.GridSearchCV(model, param_grid=params, cv=5, n_jobs=5, verbose=3)
gridSearch.fit(X_train, y_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    n

GridSearchCV(cv=5, estimator=SVC(random_state=42), n_jobs=5,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'ploy', 'rbf', 'sigmoid']},
             verbose=3)

In [14]:
y_hat = gridSearch.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

0.8449197860962567

In [11]:
import pickle
with open('./models/spect_heart_gridsearch.pkl', 'wb') as f:
    pickle.dump(gridSearch, f)

# XGBOOST

In [20]:
import xgboost

xgb = xgboost.cv({'n_estimators': 1000, 'max_depth': 6, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic', 'eval_metric': 'error'}, xgboost.DMatrix(X_train, y_train), num_boost_round=100, early_stopping_rounds=10, nfold=5)

[14:56:09] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:56:09] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:56:09] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or som

In [21]:
xgb

,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.187500,0.026146,0.4000,0.115920
1,0.184375,0.026882,0.3375,0.030619
2,0.187500,0.032775,0.3375,0.050000
3,0.178125,0.028980,0.3000,0.100000
4,0.168750,0.030298,0.2875,0.101550
5,0.175000,0.033366,0.2750,0.101550
6,0.171875,0.035630,0.2750,0.101550
7,0.168750,0.030298,0.2625,0.082916
8,0.171875,0.035630,0.2750,0.101550
9,0.159375,0.025000,0.2375,0.091856


In [23]:
model = xgboost.XGBClassifier(n_estimators=1000, max_depth=6, eta=0.1, silent=1, objective='binary:logistic', eval_metric='error')

In [24]:
model.fit(X_train, y_train)
y_hat = model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

[14:57:16] WARNING: ../src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




0.679144385026738